In [1]:
import sys
import os.path as op
import pandas as pd

basename = "/home/adrian/PhD/Tweet-Classification-Diabetes-Distress/"
path_utils = op.join(basename , "utils")
sys.path.insert(0, path_utils)

from sys_utils import load_library
from tweet_utils import tweet_vectorizer
from defines import Patterns, Emotions
from emotion_codes import UNICODE_EMOJI, EMOJI_TO_CATEGORY
from preprocess import Preprocess
prep = Preprocess()

[nltk_data] Downloading package punkt to /home/adrian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/adrian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/adrian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
tweets = pd.read_parquet("/home/adrian/PhD/Data/Tweets20190708/matching-tweets_diab_noRT-noBots_personal_noJokes_locationUS_geoCityCodeNotNull.parquet", engine="pyarrow")
print(tweets.columns)
print(tweets.shape)
tweets.head()

Index(['id', 'created_at', 'lang', 'text', 'user_name', 'user_screen_name',
       'user_followers_count', 'user_friends_count', 'user_location',
       'user_description', 'place_full_name', 'retweeted_user_name',
       'retweeted_user_screen_name', 'retweeted_user_followers_count',
       'retweeted_user_friends_count', 'retweeted_user_location',
       'retweeted_user_description', 'retweeted_place_full_name',
       'retweeted_text', 'posted_month', 'tweet_URL_USER', '__index_level_0__',
       '__index_level_1__', 'geo_id', 'geo_name', 'geo_code', 'geo_type',
       'geo_country_code', 'geo_city_code', 'geo_adm1_code', 'geo_adm2_code',
       'geo_adm3_code', 'geo_adm4_code', 'pop', '_score_', '_tags_',
       '_startIndex_', '_endIndex_'],
      dtype='object')
(167743, 38)


,id,created_at,lang,text,user_name,user_screen_name,user_followers_count,user_friends_count,user_location,user_description,...,geo_city_code,geo_adm1_code,geo_adm2_code,geo_adm3_code,geo_adm4_code,pop,_score_,_tags_,_startIndex_,_endIndex_
0,8.675721e+17,Thu May 25 02:44:51 +0000 2017,en,@kristilade It doesn't suck. I have it. It wor...,Wil Gentry,wilable70,926.0,2920.0,"Seattle, WA",Be yourself. No on else wants to be you.,...,5809844,WA,033,None,None,1.545088,221.237152,"Seattle,WA",0,2
1,1.129140e+18,Thu May 16 21:42:52 +0000 2019,en,When I was initially diagnosed with #type2diab...,Lynda Jimenez,lyndajimenez22,30.0,70.0,"Phoenix, AZ",Boymom living with LADA #diabetes. Employee a...,...,5308655,AZ,013,None,None,1.626851,262.320343,"Phoenix,AZ",0,2
2,1.102559e+18,Mon Mar 04 13:17:40 +0000 2019,en,Later Twitter it's back to sleep for a nap wok...,Deanna Porter Weick,deanna_weick,2222.0,2721.0,"Monroe, MI",Native American Potowatomi adopted out raised...,...,5002344,MI,115,None,None,1.231201,225.749695,"Monroe,MI",0,2
3,1.087817e+18,Tue Jan 22 20:58:04 +0000 2019,en,Please read this! There are people dying becau...,BTSARMYMOM,cure4t1,122.0,840.0,Rogers,"Mom, Music lover, BTS ARMY member (thanks to m...",...,4533580,OK,131,None,None,1.227495,133.306839,Rogers,0,1
4,8.641931e+17,Mon May 15 18:57:38 +0000 2017,en,My stepson was 4 or 5. Damn you kids with your...,Writing as I Go,WritingAsIGo,485.0,1195.0,"Colorado Springs, Colorado",Starting the job hunt again. Book Reviewer whi...,...,5417598,CO,041,None,None,1.505316,322.702881,"Colorado,Springs,Colorado",0,3


In [3]:

def check_emoticonEmoji_in_tweet(tweet):
    """ Check if in each tweet occurs either an emoji or emoticon """
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)

    for word in tweet: 
        match_emoticon = Patterns.EMOTICONS_PATTERN.findall(word)
        
        # if emoticon found
        if match_emoticon:
            if match_emoticon[0] is not ":" and match_emoticon[0] is not ")": return True
            
        # if emoji found    
        if word in UNICODE_EMOJI: 
            emot_cat = EMOJI_TO_CATEGORY[UNICODE_EMOJI[word]]
            if emot_cat != "": return True

        return False


def check_emotKeywords(tweet):
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = tweet.replace("nerve pain", "")
    tweet = tweet.replace("nerve damage", "")
    tweet = tweet.replace("pain of neuropathy", "")
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = [word for word in tweet if word not in ["healthcare"]]

    for word in tweet:
        for emot in Emotions.emotions_full_list:
            if emot == word: return True
    return False


tweets["emotion"] = tweets.text.map(lambda tweet: check_emoticonEmoji_in_tweet(tweet) or check_emotKeywords(tweet))

emot_tweets = tweets[tweets["emotion"] == True]
print(emot_tweets.shape)


(38316, 39)


In [5]:
# Test
for i, row in tweets[["text", "emotion"]].head().iterrows():
    print(row["text"])
    print(row["emotion"])
    print()

@kristilade It doesn't suck. I have it. It works for me. I wouldn't have found out I have diabetes without it. Now… https://t.co/kHS8R9STVC
False

When I was initially diagnosed with #type2diabetes almost 10 years ago, I was in a dark place - blaming myself, fee… https://t.co/pLtFgM2wKr
False

Later Twitter it's back to sleep for a nap woke up at 5:30am to let the puppies out. Took my diabetic meds had an e… https://t.co/7TxMvXmEja
True

Please read this! There are people dying because they can’t afford their insulin!
False

My stepson was 4 or 5. Damn you kids with your #type1diabetes and your rock and roll. https://t.co/sASXzNOG6Z
False



In [7]:
# Get most often occuring emoticons or emojis
import operator

emotico_dict = {}

for tweet in tweets.text:
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = prep.tokenize(tweet)
    for word in tweet: 
        match_emoticon = Patterns.EMOTICONS_PATTERN.findall(word)
        
        # if emoticon found
        if match_emoticon:
            if match_emoticon[0] is not ":" and match_emoticon[0] is not ")":
                if match_emoticon[0] in emotico_dict: emotico_dict[match_emoticon[0]] += 1
                else : emotico_dict[match_emoticon[0]] = 1
                
        # if emoji found    
        if word in UNICODE_EMOJI: 
            emot_cat = EMOJI_TO_CATEGORY[UNICODE_EMOJI[word]]
            if emot_cat != "": 
                if word in emotico_dict: emotico_dict[word] += 1
                else : emotico_dict[word] = 1
                
sorted_x = sorted(emotico_dict.items(), key=operator.itemgetter(1))[::-1]
print(sorted_x)

            

[('😂', 1343), ('❤', 1263), ('😭', 1202), ('💙', 926), ('🙄', 804), (':)', 655), ('😩', 643), ('🙃', 639), ('😍', 411), (':(', 335), ('😢', 314), ('😊', 284), ('😔', 273), ('😒', 269), ('😡', 262), ('dx', 229), ('😳', 226), ('🤢', 225), ('💕', 225), ('🤣', 221), ('😁', 205), ('😞', 201), ('💜', 178), ('🤗', 166), (':3', 159), (':/', 156), ('♥', 152), ('😀', 138), ('😕', 135), ('😱', 129), ('🙂', 127), ('😓', 121), ('😷', 119), ('😥', 114), (';)', 102), ('☺', 96), ('💖', 96), ('😤', 94), ('💗', 87), ('😃', 86), (':-)', 86), ('💋', 73), (':D', 72), ('😄', 68), ('😘', 68), ('):', 67), ('🙁', 64), ('😆', 61), ('😰', 55), ('💚', 55), ('💛', 53), ('😇', 52), ('😲', 49), ('😠', 48), ('Dx', 48), ('😮', 41), ('😻', 40), ('💓', 39), ('😨', 37), ('💞', 36), ('💩', 31), ('DX', 27), ('😵', 26), ('😯', 26), (':P', 24), ('(:', 22), ('0:3', 21), ('💫', 20), ('XD', 19), ('😧', 17), ('😿', 17), ('xD', 14), (':p', 11), ('🙀', 10), (":')", 10), (":'(", 10), ('dX', 9), ('=/', 9), (';D', 9), ('D:', 8), ('😺', 8), ('😸', 8), ('xd', 7), ('😹', 7), ('t.T', 7), ('=d'

In [8]:
# Get most often occuring emotional words
emojo_dict = {}

for tweet in tweets.text:
    tweet = prep.replace_hashtags_URL_USER(tweet, mode_URL="delete", mode_Mentions="delete")
    tweet = tweet.replace("nerve pain", "")
    tweet = tweet.replace("nerve damage", "")
    tweet = tweet.replace("pain of neuropathy", "")
    tweet = prep.tokenize(tweet)
    tweet = prep.remove_punctuation(tweet)
    tweet = prep.to_lowercase(tweet)
    tweet = [word for word in tweet if word not in ["healthcare"]]
    
    for word in tweet:
        for emot in Emotions.emotions_full_list:
            
            if emot == word: 
                if word in emojo_dict: emojo_dict[emot] += 1
                else : emojo_dict[emot] = 1
                    
sorted_y = sorted(emojo_dict.items(), key=operator.itemgetter(1))[::-1]
print(sorted_y)

[('good', 4834), ('love', 3044), ('feel', 2340), ('bad', 2139), ('care', 1913), ('hope', 1861), ('happy', 1806), ('sick', 1126), ('hate', 1066), ('understand', 974), ('glad', 833), ('amazing', 771), ('pain', 739), ('sad', 725), ('excited', 720), ('awesome', 698), ('depression', 538), ('poor', 491), ('shock', 487), ('worry', 485), ('alone', 431), ('funny', 387), ('worried', 362), ('anxiety', 342), ('terrible', 333), ('fear', 316), ('horrible', 307), ('attention', 294), ('loved', 294), ('stress', 289), ('suffer', 275), ('enjoy', 274), ('awful', 251), ('hurt', 250), ('shame', 239), ('surprised', 229), ('crying', 227), ('beat', 221), ('suffering', 216), ('broken', 204), ('despite', 201), ('cry', 198), ('mad', 190), ('angry', 185), ('nervous', 184), ('sadly', 161), ('outrageous', 160), ('trust', 159), ('cried', 158), ('trouble', 154), ('fail', 149), ('upset', 147), ('loving', 146), ('refuse', 143), ('affect', 135), ('catch', 134), ('sensitive', 131), ('disgusting', 129), ('annoying', 124), 

In [10]:
emot_tweets.to_parquet("/home/adrian/PhD/Data/Tweets20190708/matching-tweets_diab_noRT-noBots_personal_noJokes_locationUS_geoCityCodeNotNull_emotions.parquet", 
                       engine="pyarrow", index=False)

In [9]:
emot_tweets.head()

,id,created_at,lang,text,user_name,user_screen_name,user_followers_count,user_friends_count,user_location,user_description,...,geo_adm1_code,geo_adm2_code,geo_adm3_code,geo_adm4_code,pop,_score_,_tags_,_startIndex_,_endIndex_,emotion
1,1.129140e+18,Thu May 16 21:42:52 +0000 2019,en,When I was initially diagnosed with #type2diab...,Lynda Jimenez,lyndajimenez22,30.0,70.0,"Phoenix, AZ",Boymom living with LADA #diabetes. Employee a...,...,AZ,013,None,None,1.626851,262.320343,"Phoenix,AZ",0,2,True
2,1.102559e+18,Mon Mar 04 13:17:40 +0000 2019,en,Later Twitter it's back to sleep for a nap wok...,Deanna Porter Weick,deanna_weick,2222.0,2721.0,"Monroe, MI",Native American Potowatomi adopted out raised...,...,MI,115,None,None,1.231201,225.749695,"Monroe,MI",0,2,True
11,1.039046e+18,Mon Sep 10 07:01:45 +0000 2018,en,Ugh not ready for this 3hour glucose test tomo...,Esmeralda,EsmeBayybee,751.0,790.0,Hemet CA,Mom too a Ghoul 💚🕸\nZ.E.R.O👨‍👩‍👧‍👦,...,CA,065,None,None,1.344957,201.460098,"Hemet,CA",0,2,True
12,9.581800e+17,Tue Jan 30 03:28:18 +0000 2018,en,@hotfunkytown They also lock up diabetic suppl...,Joyce Pride,joycepride,1097.0,3466.0,"Maryville, Missouri",Farm and Ranch Real Estate Locally & Nationwid...,...,MO,147,None,None,1.199328,157.847015,"Maryville,Missouri",0,2,True
15,9.151822e+17,Tue Oct 03 11:50:09 +0000 2017,en,@I_TheeWed “Hey mom! I know you’re dreading th...,Hot Mess Momma🍁,TattoosandLace,1526.0,757.0,Illinois ✈️ Oklahoma,raised in the North now living in the South. w...,...,OK,109,None,None,1.537132,206.331146,"Illinois,Oklahoma",0,2,True
